## Step 1: Setting up environment variables

In [1]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


## Step 2: Train a model and prepare metadata for logging

In [1]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

C:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## Step 3: Log the model and its metadata to MLflow

In [ ]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

2025/03/18 14:24:37 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart 3' does not exist. Creating a new experiment.


Successfully registered model 'tracking-quickstart-3'.
2025/03/18 14:25:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart-3, version 1
Created version '1' of model 'tracking-quickstart-3'.


🏃 View run peaceful-flea-28 at: http://localhost:5000/#/experiments/578134671856356246/runs/e1b393fd75f44484a28417774ddc9534
🧪 View experiment at: http://localhost:5000/#/experiments/578134671856356246


## Step 4: Load the model as a Python Function (pyfunc) and use it for inference

In [3]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


In [4]:
# Register model explicitly
mlflow.register_model(
    model_uri=model_info.model_uri, 
    name="iris_model_registry"
)

Successfully registered model 'iris_model_registry'.
2025/03/18 11:52:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model_registry, version 1
Created version '1' of model 'iris_model_registry'.


<ModelVersion: aliases=[], creation_timestamp=1742278922117, current_stage='None', description='', last_updated_timestamp=1742278922117, name='iris_model_registry', run_id='31f1b7efa57c42f09d2b9c6c0f2223c0', run_link='', source='mlflow-artifacts:/884080327885428605/31f1b7efa57c42f09d2b9c6c0f2223c0/artifacts/iris_model', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [5]:
import mlflow
from mlflow import MlflowClient

with mlflow.start_run() as run:
    mlflow.log_param("p", 0)

client = MlflowClient()
run_info = client.get_run(run.info.run_id)
print(f"run_id: (run_info.info.run_id)")

🏃 View run rare-moose-452 at: http://127.0.0.1:5000/#/experiments/884080327885428605/runs/acfc490afe22492a9c605e83832d24b6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/884080327885428605
run_id: (run_info.info.run_id)
